In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import numpy as np
import pandas as pd
import cv2
import scipy.io as sio
import time
import glob
import matplotlib.pyplot as plt
import tqdm

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as slimNet
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
from tensorflow.python import _pywrap_tensorflow_internal


import mobilenet_v2
from tvnet.tvnet import TVNet
from file_loader import get_images_from_absdir, img_gen_from_dir, load_txt_from_path
from callbacks import *

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Get Data Directories

##### urfd

In [4]:
#
#Get urfd directories
#
daily_path = '/home/jovyan/datasets/elder-fall/UR Fall Detection Dataset(URFD)/UR_Fall_Daily'
fall_path = '/home/jovyan/datasets/elder-fall/UR Fall Detection Dataset(URFD)/UR Fall Detection Dataset'

urfd_daily = os.listdir(daily_path)
urfd_daily.remove('.DS_Store')
urfd_daily.sort(key=lambda x: int(x))

urfd_fall = os.listdir(fall_path)
urfd_fall.remove('.DS_Store')
urfd_fall.sort(key=lambda x: int(x))

In [6]:
#
#Get daily data directories
#
urfd_daily_data_dir = np.asarray([glob.glob(os.path.join(daily_path, i)+'/*rgb*') for i in urfd_daily])
urfd_daily_data_dir = np.squeeze(urfd_daily_data_dir)

#
#Get fall data directories
#
urfd_fall_data_dir = np.asarray([glob.glob(os.path.join(fall_path, i)+'/*rgb*') for i in urfd_fall])
urfd_fall_data_dir = np.squeeze(urfd_fall_data_dir)
#Sort the directories in fall data directories(cam0, cam1)
for dirs in urfd_fall_data_dir:
    dirs.sort()
#reshape urfd_fall_data_dir to one dimension
urfd_fall_data_dir = urfd_fall_data_dir.reshape(60,)

In [7]:
#
#Get label files directories
#
label_path = glob.glob('../../at073-elderfall/collection/archive/DATA/urfd_label/*')
label_path = np.asarray(label_path * 2) #there are cam0, and cam1 in urfd fall dataset
label_path.sort()

In [8]:
#
#Create dataframe for image data path and label data path
#
tmp = np.concatenate([urfd_fall_data_dir.reshape(-1, 1), label_path.reshape(-1,1)], axis=1)
df_fall = pd.DataFrame(data = tmp, columns=['img_folder', 'label_path'])

df_no_fall = pd.DataFrame(data = urfd_daily_data_dir.reshape(-1,1), columns=['img_folder'])
df_no_fall['label_path'] = 'None'

df_urfd = pd.concat([df_fall, df_no_fall], axis=0).reset_index(drop=True)
df_urfd_shuffle = df_urfd.sample(frac=1).reset_index(drop=True)

##### Multicam

In [9]:
#
#Get multicam directories
#
multicam_fall = '../../at073-elderfall/dataset/Multicam/Fall'
multicam_non_fall = '../../at073-elderfall/dataset/Multicam/Non_Fall'

In [10]:
#
#Get multicam fall data directories
#
m_fall_data_dir = np.asarray(glob.glob(multicam_fall + '/*'))
m_fall_data_dir.sort()

#
#Get multicam non_fall data directories
#
m_non_fall_data_dir = np.asarray(glob.glob(multicam_non_fall + '/*'))
m_non_fall_data_dir.sort()



In [11]:
#
#Create dataframe
#
df_multiFall = pd.DataFrame(data = m_fall_data_dir.reshape(-1,1), columns=['img_folder'])
df_multiNonFall = pd.DataFrame(data = m_non_fall_data_dir.reshape(-1,1), columns=['img_folder'])
df_multi = pd.concat([df_multiFall, df_multiNonFall], axis=0).reset_index(drop=True)
df_multi['label_path'] = 'None'
df_multi_shuffle = df_multi.sample(frac=1).reset_index(drop=True)

##### FDD

In [12]:
#
#Get FDD directories
#
fdd_fall = '../../at073-elderfall/dataset/FDD/Fall'
fdd_non_fall = '../../at073-elderfall/dataset/FDD/NoFall'

In [13]:
#
#Get FDD fall data directories
#
fdd_fall_data_dir = np.asarray(glob.glob(fdd_fall+'/*'))
fdd_fall_data_dir.sort()

#
#Get FDD non_fall data directories
#
fdd_non_fall_data_dir = np.asarray(glob.glob(fdd_non_fall+'/*'))
fdd_non_fall_data_dir.sort()

###remove the those have less than 11 frames###
frame_len = []
for imgdir in fdd_fall_data_dir:
    frame_len.append(len(glob.glob(imgdir+'/*')))
frame_len = np.asarray(frame_len)
remove_list = frame_len.argsort()[:5]

fdd_fall_data_dir = np.delete(fdd_fall_data_dir, remove_list)

In [14]:
#
#Create dataframe
#
df_fddFall = pd.DataFrame(data = fdd_fall_data_dir.reshape(-1,1), columns=['img_folder'])
df_fddNonFall = pd.DataFrame(data = fdd_non_fall_data_dir.reshape(-1,1), columns=['img_folder'])
df_fdd = pd.concat([df_fddFall, df_fddNonFall], axis=0).reset_index(drop=True)
df_fdd['label_path'] = 'None'
df_fdd_shuffle = df_fdd.sample(frac=1).reset_index(drop=True)

In [6]:
def get_fall_imgPath(folder_dir, label_dir):
    image_path_list, _ = get_images_from_absdir(folder_dir)
    label_list = load_txt_from_path(label_dir)
    img_list = image_path_list[int(label_list[0]):int(label_list[1])]
    return img_list

In [7]:
def get_imgs_labels(path_list, is_resize, resize_shape):
    frame_list = []
    
    if 'fall-' in path_list[0]:
        img_list = get_fall_imgPath(path_list[0], path_list[1])
        img_list = np.asarray(img_list)
        labels = np.zeros(img_list.shape[0]).reshape(-1,1)
    else:
        img_list, _ = get_images_from_absdir(path_list[0])
        img_list = np.asarray(img_list)
        labels = np.ones(img_list.shape[0]).reshape(-1,1)
    img_list.sort()  
     

    for idx, img in enumerate(img_list):
        frame = cv2.imread(img)
        if is_resize:
            frame = cv2.resize(frame, resize_shape, interpolation=cv2.INTER_CUBIC)
        frame_list.append(frame)
        
    return np.asarray(frame_list), labels

In [ ]:
shuffle

In [8]:
#
# Generator to generate stacks of frames
#

def imgs_labels_gen(path_df, is_resize, resize_shape):
    '''
    Images and labels generator.
    Input:
    * path_df: dataframe with images paths and labels paths as columns
    * is_resize: boolean, whether to resize the generated images
    * resize_shape: resizing shape
    '''
    while True:
        idx = random.randint(0, len(path_df))
        imgs, labels = get_imgs_labels(path_df.iloc[idx].values, is_resize, resize_shape)
        yield imgs, labels

In [9]:
def split_frames_gen(imgs, labels, L, displacement, stacks):
    for s in range(stacks):
        img1 = imgs[s*displacement:s*displacement+L-1]
        img2 = imgs[s*displacement+1:s*displacement+L]
        yield img1, img2
    
#     img1 = imgs[0:len(imgs)-1]
#     img2 = imgs[1:len(imgs)]
#     return img1, img2

### Build Model

In [10]:
#
# Graph part
#

main_graph = tf.Graph()
with main_graph.as_default():
    ###placeholder###
    with tf.name_scope('input'):
        img1 = tf.placeholder(dtype=tf.float32,
                              shape=[None, 224, 224, 3],
                              name='img1')
        img2 = tf.placeholder(dtype=tf.float32,
                              shape=[None, 224, 224, 3],
                              name='img2')
        lr = tf.placeholder(dtype=tf.float32,
                            shape=None,
                            name='learning_rate')
    ###tvnet###
    with tf.variable_scope('tvnet'):
        tvnet = TVNet()
        tvnet_loss, u1, u2 = tvnet.get_loss(x1=img1, x2=img2)
                
    ###loss###
    with tf.name_scope('loss'):
        total_loss = tvnet_loss
        
    ###train###
    with tf.name_scope('train_op'):
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        grad_var_pairs = optimizer.compute_gradients(total_loss)
        update = optimizer.apply_gradients(grad_var_pairs)
            
    ###others###
    var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    all_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### Callbacks

In [18]:
model_dict = {
    'model_name' : 'pre_train_tvnet',
    'reduce_lr' : ReduceLROnPlateau(lr=1e-3, factor=0.5, patience=5),
    'earlystop' : EarlyStopping(min_delta = 1e-4, patience= 10),
    'checkpoint' : Model_checkpoint(os.path.join('./pre_train_model_2', 'pre_train_tvnet')),
    'train_batch_log' : History(['loss']),
    'val_batch_log' : History(['loss']),
    'history' : {
        'train_loss':[],
        'val_loss':[]
    },
    'testing' : {
        'y_true' : [],
        'y_pred' : [],
        'files'   : []
    }
}

callback_dict = {
    'on_session_begin':[], # start of a session
    'on_batch_begin':[], # start of a training batch
    'on_batch_end':[], # end of a training batch
    'on_epoch_begin':[], # start of a epoch
    'on_epoch_end':[
        model_dict['reduce_lr'],
        model_dict['checkpoint']
    ], # end of a epoch
    'on_session_end':[] # end of a session
}
callback_manager = Run_collected_functions(callback_dict)

In [19]:
epochs = 500
L = 11
displacement = 6

In [ ]:
# load_var_list = [v for v in var_list if 'MobilenetV2' in v.name][3:]
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True), allow_soft_placement=True, log_device_placement=True)
with tf.Session(graph=main_graph, config=config) as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    for epoch in tqdm.tqdm_notebook(range(epochs)):
        if epoch % 100 == 0:
                train_gen = imgs_labels_gen(path_df=df_shuffle, is_resize=True, resize_shape=(224,224))
        tmp_loss = 0
        X_batch, y_batch = next(train_gen)
        stacks = (len(X_batch) - L) // displacement
        img_gen = split_frames_gen(imgs=X_batch, L=L, displacement=displacement, stacks=stacks)
        for s in range(stacks):
            imgs_1, imgs_2 = next(img_gen)
            start_time = time.time()
            _, loss_value = sess.run([update, total_loss], feed_dict={img1: imgs_1, img2: imgs_2, lr: model_dict['reduce_lr'].lr})
            print('Time of training a video: {}'.format(time.time() - start_time))
            print('Loss: {}'.format(loss_value))
            tmp_loss += loss_value
        # Save loss
        model_dict['history']['train_loss'].append(tmp_loss/stacks)
        # call back
        callback_manager.run_on_epoch_end(val_loss=model_dict['history']['train_loss'][-1],
                                          sess = sess,
                                          saver = saver,
                                          nth_epoch = epoch)
        
        

Time of generating: 0.6011962890625
Time of training a video: 148.98572993278503
Loss: 0.5728211402893066
Time of training a video: 0.7807698249816895
Loss: 0.6178855299949646
Time of training a video: 0.7798223495483398
Loss: 0.665643572807312
